In [13]:
import os
import numpy as np
from skimage import io, transform, exposure

import napari

from scripts.utils.image_utils import load_anatomy_stack, save_array_as_hyperstack_tiff
from scripts.sample_db import SampleDB


# Load the sample database
db_path = r'\\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\sample_db.csv'
sample_db = SampleDB()
sample_db.load(db_path)
print(sample_db)

# Get the sample
sample_id = '20220118_RM0012_124hpf_fP8_f2'
exp = sample_db.get_sample(sample_id)

# Print the paths to verify
print("Anatomy path:", exp.paths.anatomy_path)

# Assuming the LM stack is in the anatomy/raw folder
lm_stack_name = f'{exp.sample.id}_anatomyGFRF_001_.tif'
lm_stack_path = os.path.join(exp.paths.anatomy_path, 'raw', lm_stack_name)

# Use noise2void stack if available
n2v_stack_path = os.path.join(exp.paths.anatomy_path, 'processed', f'{exp.sample.id}_anatomyGFRF_001_n2v.tif')
if os.path.exists(n2v_stack_path):
    print("Using noise2void stack")
    lm_stack = load_anatomy_stack(n2v_stack_path)
else:
    print("Using original stack")
    lm_stack = load_anatomy_stack(lm_stack_path)

print("LM stack shape:", lm_stack.shape)

# Flip horizontally
print("Flipping stack")
lm_stack_flip = np.flip(lm_stack, axis=3)

# Upsample the stack
print("Upsampling stack")
upsample_factor=(1,2.5, 1, 1)
lm_stack_upsampled = transform.rescale(lm_stack_flip, upsample_factor, order=1, mode='reflect', anti_aliasing=True, preserve_range=False)

#Apply CLAHE
print("Applying CLAHE")
lm_stack_clahe = lm_stack_upsampled
lm_stack_clahe[0]=exposure.equalize_adapthist(lm_stack_upsampled[0])
lm_stack_clahe[1]=exposure.equalize_adapthist(lm_stack_upsampled[1])


# Preprocess the LM stack
output_folder = os.path.join(exp.paths.anatomy_path, 'processed' )
os.makedirs(output_folder, exist_ok=True)

viewer = napari.Viewer()
viewer.add_image(lm_stack)
viewer.add_image(lm_stack_flip)
viewer.add_image(lm_stack_upsampled)
viewer.add_image(lm_stack_clahe)

SampleDB(sample_ids=['20220426_RM0008_130hpf_fP1_f3', '20220118_RM0012_124hpf_fP8_f2', '20220427_RM0008_126hpf_fP3_f3'])
Anatomy path: \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\anatomy
Using original stack
LM stack shape: (2, 150, 512, 512)
Flipping stack
Upsampling stack
Applying CLAHE


D:\montruth\PycharmProjects\samplePy\.venv\lib\site-packages\napari\plugins\_plugin_manager.py:555: UserWarning: Plugin 'napari-animated-gif-io' has already registered a function widget 'save as animated gif' which has now been overwritten
  warn(message=warn_message)


<Image layer 'lm_stack_clahe' at 0x27be1279b40>

In [11]:
import tifffile
def save_array_as_hyperstack_tiff(path, array):
    array_reshaped = array.transpose(1, 0, 2, 3).astype(np.float32)
    #array = array.astype(np.float32)
    # Save as TIFF with tifffile
    tifffile.imwrite(path, array_reshaped, imagej=True)
# Save the preprocessed stack
output_file = os.path.join(output_folder,"flipped_upsampled_"+lm_stack_name)
save_array_as_hyperstack_tiff(output_file, lm_stack_upsampled)

print(f"Preprocessed stack saved to: {output_file}")

Preprocessed stack saved to: \\tungsten-nas.fmi.ch\tungsten\scratch\gfriedri\montruth\2P_RawData\2022-01-18\f2\anatomy\processed\flipped_upsampled20220118_RM0012_124hpf_fP8_f2_anatomyGFRF_001_.tif
